In [1]:
from pywgraph import *

In [2]:
grafo = WeightedDirectedGraph.from_dict(
    {
        "A": {"B": 2, "F": 1},
        "B": {"C": 1, "A":1},
        "C": {"A":1, "D":1},
        "D": {"A":1, "E":1},
        "E": {"B":1, "F": 1},
        "F":{"E":1, "A":1}
    }
)

In [7]:
set(grafo.get_node_cycles("A"))

{Cycle(['A']),
 Cycle(['A', 'B']),
 Cycle(['A', 'B', 'C']),
 Cycle(['A', 'B', 'C', 'D']),
 Cycle(['A', 'B', 'C', 'D', 'E', 'F']),
 Cycle(['A', 'F']),
 Cycle(['A', 'F', 'E', 'B']),
 Cycle(['A', 'F', 'E', 'B', 'C']),
 Cycle(['A', 'F', 'E', 'B', 'C', 'D'])}

In [8]:
set(grafo.find_paths("A","A", 1, {"A":2}))

{['A'],
 ['A', 'B', 'A'],
 ['A', 'B', 'C', 'A'],
 ['A', 'B', 'C', 'D', 'A'],
 ['A', 'B', 'C', 'D', 'E', 'F', 'A'],
 ['A', 'F', 'A'],
 ['A', 'F', 'E', 'B', 'A'],
 ['A', 'F', 'E', 'B', 'C', 'A'],
 ['A', 'F', 'E', 'B', 'C', 'D', 'A']}

In [10]:
def graph() -> WeightedDirectedGraph:
    _dict = {
        "A":{"B":1},
        "B":{"C":1},
        "C":{},
        "Z":{}
    }
    return WeightedDirectedGraph.from_dict(_dict).add_reverse_edges()

graph().find_paths("A", "A", general_max_visitations=2)

[['A'], ['A', 'B', 'A'], ['A', 'B', 'C', 'B', 'A']]

In [11]:
graph().get_node_cycles("A")

[Cycle(['A']), Cycle(['A', 'B'])]

In [6]:
Cycle(["A","B","A"]) == Path(["A","B","A"])

True

In [ ]:
sorted(list(grafo.get_node_cycles("A")), key=len)

In [ ]:
class PathExplorer(list[str]):

    def __init__(self, path: list[str] = [], visited: set[str] = set()):
        self._path = path
        self._visited = visited

    @property
    def path(self) -> list[str]:
        return self._path
    
    @property
    def visited(self) -> set[str]:
        return self._visited
    
    def __hash__(self) -> int:
        return hash((tuple(self.path), tuple(self.visited)))
    
    def __eq__(self, other: object) -> bool:
        if isinstance(other, PathExplorer):
            return (self.path, self.visited) == (other.path, other.visited)
        return False
    
    def __repr__(self) -> str:
        return f"PathExplorer({self._path}, {self._visited})"
    
    def __len__(self) -> int:
        return len(self._path)

In [ ]:
def _cycle_aux(
    grafo: WeightedDirectedGraph, explorer: PathExplorer, target: str
) -> tuple[list, str] | tuple[Cycle, str]:
    current_node = explorer.path[-1]
    if current_node == target:
        return Cycle(explorer.path[:-1]), "cycle"
    
    children = grafo.children(current_node)
    unexplored_nodes = children - explorer.visited
    if not unexplored_nodes:
        return [], "dead explorer"
    
    updated_visited = explorer.visited | {current_node}
    new_explorers = [
        PathExplorer(explorer.path + [node], updated_visited)
        for node in unexplored_nodes
    ]
    return new_explorers, "continue"

In [ ]:
def _get_node_cycles(grafo: WeightedDirectedGraph, start: str) -> list[list[str]]:
    first_children = grafo.children(start)
    if not first_children:
        return []

    explorers: list[PathExplorer] = [
        PathExplorer([start, child]) for child in first_children
    ]
    cycles: list[Cycle] = []

    while explorers: 
        result, state = _cycle_aux(grafo, explorers.pop(0), start)                              
        if state == "cycle":
            cycles.append(result)
        elif state == "continue":
            new_explorers = list(set(result) - set(explorers))
            explorers.extend(new_explorers)

    return cycles

In [ ]:
def _path_aux(
    grafo: WeightedDirectedGraph, explorer: PathExplorer, target: str
) -> tuple[list, str] | tuple[Cycle, str]:
    current_node = explorer.path[-1]
    if current_node == target:
        return explorer.path, "path"
    
    children = grafo.children(current_node)
    unexplored_nodes = children - explorer.visited
    if not unexplored_nodes:
        return [], "dead explorer"
    
    updated_visited = explorer.visited | {current_node}
    new_explorers = [
        PathExplorer(explorer.path + [node], updated_visited)
        for node in unexplored_nodes
    ]
    return new_explorers, "continue"

In [ ]:
def _find_all_paths(grafo: WeightedDirectedGraph, start: str, end: str) -> list[list[str]]:
    first_children = grafo.children(start)
    if not first_children:
        return []

    explorers: list[PathExplorer] = [
        PathExplorer([start, child]) for child in first_children
    ]
    all_paths: list[list[str]] = []

    while explorers:
        result, state = _path_aux(grafo, explorers.pop(0), end)                  
        if state == "path":
            all_paths.append(result)
        elif state == "continue":
            new_explorers = list(set(result) - set(explorers))
            explorers.extend(new_explorers)

    return all_paths

In [ ]:
grafo.get_node_cycles("A")

In [ ]:
_find_all_paths(grafo, "A", "F")

In [ ]:
grafo.get_node_cycles("B")

In [ ]:
grafo.get_node_cycles("C")

In [ ]:
grafo.get_node_cycles("D")

In [ ]:
grafo.get_node_cycles("E")

In [ ]:
grafo.get_node_cycles("F")

In [ ]:
grafo.cycles

In [ ]:
def vector_group_multiplication() -> Group:
    group = Group(
        name="Vectors of dimension 2 with multiplication",
        identity=np.ones(2),
        operation=lambda x, y: x * y,
        inverse_function=lambda x: 1 / x,
        hash_function=lambda x: hash(tuple(x)),
    )
    return group


def vector_group_addition() -> Group:
    group = Group(
        name="Vectors of dimension 2 with addition",
        identity=np.zeros(2),
        operation=lambda x, y: x + y,
        inverse_function=lambda x: -x,
        hash_function=lambda x: hash(tuple(x)),
    )
    return group


_array_dict_graph: dict[str, dict[str, Any]] = {
    "A": {
        "B": np.array([1.0, 2.0]),
        "C": np.array([3.0, 4.0]),
    },
    "B": {
        "C": np.array([-5.0, 1.3]),
        "D": np.array([2.0, 1.0]),
    },
    "C": {
        "D": np.array([-1.0, 1.0]),
    },
    "D": {},
    "Z": {},
}


def multiplication_graph() -> WeightedDirectedGraph:
    graph = WeightedDirectedGraph.from_dict(
        _array_dict_graph, vector_group_multiplication()
    )
    return graph


def addition_graph() -> WeightedDirectedGraph:
    graph = WeightedDirectedGraph.from_dict(_array_dict_graph, vector_group_addition())
    return graph

In [ ]:
addition_graph().add_reverse_edges().is_conmutative

In [ ]:
complete_graph = addition_graph().add_reverse_edges()
group = complete_graph.group
identity = group.identity
bad = [
    cycle
    for cycle in complete_graph.cycles
    if not group.equal(complete_graph.path_weight(cycle), identity)
]
bad 

In [ ]:
[
    complete_graph.path_weight(cycle)
    for cycle in bad
]

In [ ]:
complete_graph

In [ ]:
todos = set()
for node in complete_graph.nodes: 
    todos.update(complete_graph.get_node_cycles(node))

todos

In [ ]:
[cycle for cycle in complete_graph.cycles]

In [ ]:
grafo = WeightedDirectedGraph.from_dict(
    {
        "A": {"B": np.array([1, 2.0]), "C": np.array([3.0, 4.0])},
        "B": {"C": np.array([2.0, 2.0])},
        "C": {},
    },
    vector_group_addition(),
)

In [ ]:
grafo.add_reverse_edges().cycles